In [1]:
import os, time, pickle, random, time
from datetime import datetime
import numpy as np
from time import localtime, strftime
import logging, scipy

import tensorflow as tf
import tensorlayer as tl
from model import SRGAN_g, SRGAN_d, Vgg19_simple_api
from utils import *
from config import config, log_config

In [24]:
checkpoint_dir = "checkpoint"
save_dir = 'sample_dir'
path = './test_img/DIV2K_valid_LR_bicubic/X4'

In [8]:
valid_lr_img_list = sorted(tl.files.load_file_list(path=path, regx='.*.png', printable=False))

valid_lr_imgs = tl.vis.read_images(valid_lr_img_list, \
                        path=path, n_threads=32)

valid_lr_imgs = tl.vis.read_images(valid_lr_img_list, \
                        path=path, n_threads=32)

[TL] read 32 from ./test_img/DIV2K_valid_LR_bicubic/X4
[TL] read 64 from ./test_img/DIV2K_valid_LR_bicubic/X4
[TL] read 96 from ./test_img/DIV2K_valid_LR_bicubic/X4
[TL] read 100 from ./test_img/DIV2K_valid_LR_bicubic/X4


In [12]:
imid = 50  # 0: 企鹅  81: 蝴蝶 53: 鸟  64: 古堡
valid_lr_img = valid_lr_imgs[imid]
valid_lr_img = (valid_lr_img / 127.5) - 1

In [13]:
t_image = tf.placeholder('float32', [1, None, None, 3], name='input_image')
net_g = SRGAN_g(t_image, is_train=False, reuse=False)

sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=False))
tl.layers.initialize_global_variables(sess)
tl.files.load_and_assign_npz(sess=sess, name=checkpoint_dir + '/g_srgan.npz', network=net_g)

[TL] InputLayer  SRGAN_g/in: (1, ?, ?, 3)
[TL] Conv2d SRGAN_g/n64s1/c: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d SRGAN_g/n64s1/c1/0: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b1/0: decay: 0.900000 epsilon: 0.000010 act: relu is_train: False
[TL] Conv2d SRGAN_g/n64s1/c2/0: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b2/0: decay: 0.900000 epsilon: 0.000010 act: No Activation is_train: False
[TL] ElementwiseLayer SRGAN_g/b_residual_add/0: size: (1, ?, ?, 64) fn: add
[TL] Conv2d SRGAN_g/n64s1/c1/1: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b1/1: decay: 0.900000 epsilon: 0.000010 act: relu is_train: False
[TL] Conv2d SRGAN_g/n64s1/c2/1: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b

[TL] Conv2d SRGAN_g/n64s1/c/m: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b/m: decay: 0.900000 epsilon: 0.000010 act: No Activation is_train: False
[TL] ElementwiseLayer SRGAN_g/add3: size: (1, ?, ?, 64) fn: add
[TL] Conv2d SRGAN_g/n256s1/1: n_filter: 256 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] SubpixelConv2d  SRGAN_g/pixelshufflerx2/1: scale: 2 n_out_channel: 64 act: relu
[TL] Conv2d SRGAN_g/n256s1/2: n_filter: 256 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] SubpixelConv2d  SRGAN_g/pixelshufflerx2/2: scale: 2 n_out_channel: 64 act: relu
[TL] Conv2d SRGAN_g/out: n_filter: 3 filter_size: (1, 1) strides: (1, 1) pad: SAME act: tanh


In [17]:
out = sess.run(net_g.outputs, {t_image: [valid_lr_img]})

In [18]:
out.shape

(1, 1344, 2040, 3)

In [33]:
tl.vis.save_image(out[0], 'output' + '/valid_gen.png')

In [35]:
tl.vis.save_image(valid_lr_img, 'output' + '/valid_ori.png')

In [28]:
valid_lr_img.shape

(336, 510, 3)

In [11]:
from PIL import Image
import numpy as np

img = Image.fromarray(valid_lr_imgs[50], 'RGB')
img.save('my.png')
img.show()